In [1]:
!pip install tigramite

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.6/309.6 kB 4.9 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
from scipy import stats
from tigramite import plotting as tp
import matplotlib.pyplot as plt
from pathlib import Path

In [15]:
# D3C.ipynb

# Nonlinear time series generator with mixed errors, irregular sampling,
# mixed missing data patterns, and a confounder

def get_nonlinear_equations_with_confounder(n_vars, max_lag):
    """Get nonlinear equations with confounder U (max 2 edges from U)"""
    if n_vars == 4:
        if max_lag == 2:
            return [
                "X4[t] = 0.25 * cos(X1[t-2] * pi/2) + trend4[t] + season4[t] + 0.3 * U[t]^2 + e4",
                "X3[t] = 0.35 * (X4[t])^2 + trend3[t] + season3[t] + e3",
                "X2[t] = 0.3 * sin(X3[t-1] * pi/2) + trend2[t] + season2[t] + e2",
                "X1[t] = 0.4 * (X2[t])^3 + trend1[t] + season1[t] + 0.5 * U[t]^2 + e1",
                "U[t] = eU"
            ]
        elif max_lag == 3:
            return [
                "X4[t] = 0.25 * cos(X1[t-2] * pi/2) + trend4[t] + season4[t] + 0.3 * U[t]^2 + e4",
                "X3[t] = 0.35 * (X4[t])^2 + 0.2 * cos(X2[t-3] * pi/2) + trend3[t] + season3[t] + e3",
                "X2[t] = 0.3 * sin(X3[t-1] * pi/2) + trend2[t] + season2[t] + e2",
                "X1[t] = 0.4 * (X2[t])^3 + trend1[t] + season1[t] + 0.5 * U[t]^2 + e1",
                "U[t] = eU"
            ]
        elif max_lag == 4:
            return [
                "X4[t] = 0.25 * cos(X1[t-4] * pi/2) + trend4[t] + season4[t] + 0.3 * U[t]^2 + e4",
                "X3[t] = 0.35 * (X4[t])^2 + 0.2 * cos(X2[t-3] * pi/2) + trend3[t] + season3[t] + e3",
                "X2[t] = 0.3 * sin(X3[t-1] * pi/2) + trend2[t] + season2[t] + e2",
                "X1[t] = 0.4 * (X2[t])^3 + trend1[t] + season1[t] + 0.5 * U[t]^2 + e1",
                "U[t] = eU"
            ]
    elif n_vars == 6:
        if max_lag == 2:
            return [
                "X6[t] = 0.45 * sin(X5[t] * pi/2) + trend6[t] + season6[t] + 0.4 * U[t]^2 + e6",
                "X5[t] = 0.3 * cos(X4[t-1] * pi/2) + trend5[t] + season5[t] + e5",
                "X4[t] = 0.25 * cos(X1[t-2] * pi/2) + trend4[t] + season4[t] + e4",
                "X3[t] = 0.35 * (X4[t])^2 + trend3[t] + season3[t] + e3",
                "X2[t] = 0.3 * sin(X3[t-1] * pi/2) + trend2[t] + season2[t] + e2",
                "X1[t] = 0.4 * (X2[t])^3 + trend1[t] + season1[t] + 0.5 * U[t]^2 + e1",
                "U[t] = eU"
            ]
        elif max_lag == 3:
            return [
                "X6[t] = 0.45 * sin(X5[t] * pi/2) + trend6[t] + season6[t] + 0.4 * U[t]^2 + e6",
                "X5[t] = 0.3 * cos(X4[t-1] * pi/2) + trend5[t] + season5[t] + e5",
                "X4[t] = 0.25 * cos(X1[t-2] * pi/2) + trend4[t] + season4[t] + e4",
                "X3[t] = 0.35 * (X4[t])^2 + 0.2 * cos(X2[t-3] * pi/2) + trend3[t] + season3[t] + e3",
                "X2[t] = 0.3 * sin(X3[t-1] * pi/2) + trend2[t] + season2[t] + e2",
                "X1[t] = 0.4 * (X2[t])^3 + trend1[t] + season1[t] + 0.5 * U[t]^2 + e1",
                "U[t] = eU"
            ]
        elif max_lag == 4:
            return [
                "X6[t] = 0.45 * sin(X5[t] * pi/2) + trend6[t] + season6[t] + 0.4 * U[t]^2 + e6",
                "X5[t] = 0.3 * cos(X4[t-1] * pi/2) + trend5[t] + season5[t] + e5",
                "X4[t] = 0.25 * cos(X1[t-4] * pi/2) + trend4[t] + season4[t] + e4",
                "X3[t] = 0.35 * (X4[t])^2 + 0.2 * cos(X2[t-3] * pi/2) + trend3[t] + season3[t] + e3",
                "X2[t] = 0.3 * sin(X3[t-1] * pi/2) + trend2[t] + season2[t] + e2",
                "X1[t] = 0.4 * (X2[t])^3 + trend1[t] + season1[t] + 0.5 * U[t]^2 + e1",
                "U[t] = eU"
            ]
    elif n_vars == 8:
        if max_lag == 2:
            return [
                "X8[t] = 0.4 * sin(X7[t] * pi/2) + trend8[t] + season8[t] + 0.35 * U[t]^2 + e8",
                "X7[t] = 0.35 * cos(X6[t-1] * pi/2) + trend7[t] + season7[t] + e7",
                "X6[t] = 0.45 * sin(X5[t] * pi/2) + trend6[t] + season6[t] + e6",
                "X5[t] = 0.3 * cos(X4[t-1] * pi/2) + trend5[t] + season5[t] + e5",
                "X4[t] = 0.25 * cos(X1[t-2] * pi/2) + trend4[t] + season4[t] + e4",
                "X3[t] = 0.35 * (X4[t])^2 + trend3[t] + season3[t] + e3",
                "X2[t] = 0.3 * sin(X3[t-1] * pi/2) + trend2[t] + season2[t] + e2",
                "X1[t] = 0.4 * (X2[t])^3 + trend1[t] + season1[t] + 0.5 * U[t]^2 + e1",
                "U[t] = eU"
            ]
        elif max_lag == 3:
            return [
                "X8[t] = 0.4 * sin(X7[t] * pi/2) + trend8[t] + season8[t] + 0.35 * U[t]^2 + e8",
                "X7[t] = 0.35 * cos(X6[t-1] * pi/2) + trend7[t] + season7[t] + e7",
                "X6[t] = 0.45 * sin(X5[t] * pi/2) + trend6[t] + season6[t] + e6",
                "X5[t] = 0.3 * cos(X4[t-1] * pi/2) + trend5[t] + season5[t] + e5",
                "X4[t] = 0.25 * cos(X1[t-2] * pi/2) + trend4[t] + season4[t] + e4",
                "X3[t] = 0.35 * (X4[t])^2 + 0.2 * cos(X2[t-3] * pi/2) + trend3[t] + season3[t] + e3",
                "X2[t] = 0.3 * sin(X3[t-1] * pi/2) + trend2[t] + season2[t] + e2",
                "X1[t] = 0.4 * (X2[t])^3 + trend1[t] + season1[t] + 0.5 * U[t]^2 + e1",
                "U[t] = eU"
            ]
        elif max_lag == 4:
            return [
                "X8[t] = 0.4 * sin(X7[t] * pi/2) + trend8[t] + season8[t] + 0.35 * U[t]^2 + e8",
                "X7[t] = 0.35 * cos(X6[t-1] * pi/2) + trend7[t] + season7[t] + e7",
                "X6[t] = 0.45 * sin(X5[t] * pi/2) + trend6[t] + season6[t] + e6",
                "X5[t] = 0.3 * cos(X4[t-1] * pi/2) + trend5[t] + season5[t] + e5",
                "X4[t] = 0.25 * cos(X1[t-4] * pi/2) + trend4[t] + season4[t] + e4",
                "X3[t] = 0.35 * (X4[t])^2 + 0.2 * cos(X2[t-3] * pi/2) + trend3[t] + season3[t] + e3",
                "X2[t] = 0.3 * sin(X3[t-1] * pi/2) + trend2[t] + season2[t] + e2",
                "X1[t] = 0.4 * (X2[t])^3 + trend1[t] + season1[t] + 0.5 * U[t]^2 + e1",
                "U[t] = eU"
            ]
    return []

def extract_causal_links(equations):
    """Extract causal links from nonlinear equations including confounder"""
    links = {}
    for eq in equations:
        if '=' not in eq or eq.startswith('U['):
            continue

        left, right = eq.split('=')
        target = left.split('[')[0]

        terms = right.split('+')
        for term in terms:
            term = term.strip()
            if not ('X' in term or 'U' in term) or term.startswith('e'):
                continue

            coef = float(term.split('*')[0])

            # Handle U terms
            if 'U[' in term:
                if '^' in term:
                    power = int(term.split('^')[1])
                    links[('U', 0, target, f'power{power}')] = coef
                else:
                    links[('U', 0, target, 'linear')] = coef
                continue

            # Handle X terms
            if 'cos(' in term:
                func = 'cos'
                source = term.split('X')[1].split('[')[0]
            elif 'sin(' in term:
                func = 'sin'
                source = term.split('X')[1].split('[')[0]
            elif '^' in term:
                func = f'power{term.split("^")[1]}'
                source = term.split('X')[1].split('[')[0]
            else:
                func = 'linear'
                source = term.split('X')[1].split('[')[0]

            if '[t-' in term:
                lag = int(term.split('-')[1].split(']')[0])
            else:
                lag = 0

            links[(f'X{source}', lag, target, func)] = coef

    return links

class NonlinearTimeSeriesGeneratorMixedMissingConfounded:
    def __init__(self, noise_mix_ratio=0.5, noise_params={'scale': 0.1},
                 mcar_prob=0.05, block_size_range=(5, 20), block_prob=0.05,
                 trend_strength=0.01, seasonal_strength=0.5, seasonal_period=12,
                 random_state=None):
        self.noise_mix_ratio = noise_mix_ratio
        self.noise_params = noise_params
        self.mcar_prob = mcar_prob
        self.block_size_range = block_size_range
        self.block_prob = block_prob
        self.trend_strength = trend_strength
        self.seasonal_strength = seasonal_strength
        self.seasonal_period = seasonal_period
        self.random_state = random_state
        if random_state is not None:
            np.random.seed(random_state)

    def generate_mixed_noise(self, size, include_confounder=False):
        """
        Generate mixed Gaussian and Laplace noise

        Parameters:
        - size: number of noise terms to generate
        - include_confounder: if True, adds an extra noise term for the confounder
        """
        total_size = size + 1 if include_confounder else size
        mask = np.random.random(total_size) < self.noise_mix_ratio
        gaussian_noise = np.random.normal(0, self.noise_params['scale'], size=total_size)
        laplace_noise = np.random.laplace(0, self.noise_params['scale'], size=total_size)
        mixed_noise = np.where(mask, gaussian_noise, laplace_noise)

        return mixed_noise if include_confounder else mixed_noise[:size]

    def generate_irregular_timestamps(self, n_points, total_time, min_gap=0.1):
        """Generate irregular sampling times"""
        times = np.zeros(n_points)
        times[0] = np.random.uniform(0, min_gap)

        for i in range(1, n_points):
            gap = np.random.exponential(scale=(total_time-times[i-1])/(n_points-i))
            times[i] = times[i-1] + max(gap, min_gap)

            if times[i] > total_time:
                times = times * (total_time / times[i])

        return times

    def generate_missing_mask(self, n_points, n_vars):
        """Generate MCAR and MAR/NMAR block missingness masks"""
        missing_mask = np.zeros((n_points, n_vars), dtype=bool)

        # Generate MCAR mask
        mcar_mask = np.random.random((n_points, n_vars)) < self.mcar_prob

        # Generate MAR/NMAR block missing mask
        current_block = False
        block_remaining = 0
        for t in range(n_points):
            if not current_block and np.random.random() < self.block_prob:
                current_block = True
                block_remaining = np.random.randint(*self.block_size_range)
                n_affected = np.random.randint(1, n_vars + 1)
                affected_vars = np.random.choice(n_vars, size=n_affected, replace=False)
                for _ in range(block_remaining):
                    if t + _ < n_points:
                        missing_mask[t + _, affected_vars] = True
            if current_block:
                block_remaining -= 1
                if block_remaining <= 0:
                    current_block = False

        # Combine MCAR and block missing masks
        missing_mask |= mcar_mask

        return missing_mask

    def generate_trend(self, n_points, var_idx):
        """Generate deterministic trend component"""
        trend_modifier = (var_idx + 1) * 0.5
        t = np.arange(n_points)
        return self.trend_strength * trend_modifier * t

    def generate_seasonality(self, timestamps, var_idx):
        """Generate seasonal component using continuous time"""
        phase_shift = 2 * np.pi * var_idx / 8
        season1 = np.sin(2 * np.pi * timestamps / self.seasonal_period + phase_shift)
        season2 = 0.5 * np.cos(4 * np.pi * timestamps / self.seasonal_period + phase_shift)
        return self.seasonal_strength * (season1 + season2)

    def find_nearest_lag_idx(self, timestamps, current_idx, lag_time):
        """Find index of nearest available past observation for given lag"""
        target_time = timestamps[current_idx] - lag_time
        past_timestamps = timestamps[:current_idx]
        if len(past_timestamps) == 0:
            return 0
        return (np.abs(past_timestamps - target_time)).argmin()

    def evaluate_term(self, term, var_values, X, U, t, timestamps, lag_indices):
        """Evaluate a single term in the equation with irregular sampling and confounder"""
        parts = term.split('*')
        coef = float(parts[0].strip())
        expr = parts[1].strip()

        # Handle U terms first
        if expr.startswith('U['):
            if expr == 'U[t]':
                if '^' in expr:
                    power = int(expr.split('^')[1])
                    return coef * (U[t] ** power)
                return coef * U[t]
            # U doesn't actually have lagged terms in our equations
            return coef * U[t]

        # Handle autocorrelation terms
        if '[t-' in expr and not any(func in expr for func in ['cos', 'sin', '^']):
            var_idx = int(expr[1]) - 1
            lag = int(expr.split('-')[1].split(']')[0])
            lag_idx = lag_indices[lag-1]
            return coef * X[lag_idx, var_idx]

        # Handle nonlinear terms
        if 'cos(' in expr:
            inner = expr.split('cos(')[1].split(')')[0]
            var_idx = int(inner.split('X')[1].split('[')[0]) - 1
            if '[t-' in inner:
                lag = int(inner.split('-')[1].split(']')[0])
                lag_idx = lag_indices[lag-1]
                value = X[lag_idx, var_idx]
            else:
                value = X[t, var_idx]
            return coef * np.cos(value * np.pi/2)
        elif 'sin(' in expr:
            inner = expr.split('sin(')[1].split(')')[0]
            var_idx = int(inner.split('X')[1].split('[')[0]) - 1
            if '[t-' in inner:
                lag = int(inner.split('-')[1].split(']')[0])
                lag_idx = lag_indices[lag-1]
                value = X[lag_idx, var_idx]
            else:
                value = X[t, var_idx]
            return coef * np.sin(value * np.pi/2)
        elif '^' in expr:
            power = int(expr.split('^')[1])
            var_idx = int(expr.split('X')[1].split('[')[0]) - 1
            if '[t-' in expr:
                lag = int(expr.split('-')[1].split(']')[0])
                lag_idx = lag_indices[lag-1]
                value = X[lag_idx, var_idx]
            else:
                value = X[t, var_idx]
            return coef * (value ** power)

        # Default linear case
        var_idx = int(expr.split('X')[1].split('[')[0]) - 1
        if '[t-' in expr:
            lag = int(expr.split('-')[1].split(']')[0])
            lag_idx = lag_indices[lag-1]
            value = X[lag_idx, var_idx]
        else:
            value = X[t, var_idx]
        return coef * value

    def generate_equations(self, t, X, U, trends, seasonality, n_vars, max_lag, timestamps, lag_indices):
        noise = self.generate_mixed_noise(n_vars, include_confounder=True)  # Include confounder noise
        equations = get_nonlinear_equations_with_confounder(n_vars, max_lag)
        var_values = {}

        # Generate U first (confounder)
        U[t] = noise[-1]  # Last noise term is for U
        var_values['U'] = U[t]
        for eq in equations:
            if '=' not in eq or eq.startswith('U['):
                continue

            left, right = eq.split('=')
            var_name = left.split('[')[0]
            var_idx = int(var_name[1:]) - 1

            terms = [term.strip() for term in right.split('+')]
            value = 0

            for term in terms:
                if term.startswith('e'):  # Noise term
                    value += noise[var_idx]
                elif term.startswith('trend'):
                    value += trends[var_idx][t]
                elif term.startswith('season'):
                    value += seasonality[var_idx][t]
                else:
                    value += self.evaluate_term(term, var_values, X, U, t, timestamps, lag_indices)

            X[t, var_idx] = value
            var_values[var_name] = value

    def generate_multivariate_ts(self, n_points, n_vars, max_lag, total_time=100, min_gap=0.1):
        """Generate multivariate time series with mixed noise, irregular sampling, mixed missing patterns, and confounder"""
        # Generate irregular timestamps
        timestamps = self.generate_irregular_timestamps(n_points, total_time, min_gap)

        # Initialize arrays
        X = np.zeros((n_points, n_vars))
        U = np.zeros(n_points)  # Array for confounder U

        # Generate trends and seasonality components
        trends = [self.generate_trend(n_points, i) for i in range(n_vars)]
        seasonality = [self.generate_seasonality(timestamps, i) for i in range(n_vars)]

        # Initialize first steps with noise
        for i in range(max_lag):
            X[i] = self.generate_mixed_noise(n_vars)  # Noise for variables
            U[i] = self.generate_mixed_noise(1, include_confounder=True)[-1]  # Noise for U
            # Add trend and seasonality to initial values
            for j in range(n_vars):
                X[i, j] += trends[j][i] + seasonality[j][i]
        # Generate time series
        for t in range(max_lag, n_points):
            mean_diff = np.mean(np.diff(timestamps))
            lag_indices = [self.find_nearest_lag_idx(timestamps, t, i * mean_diff)
                         for i in range(1, max_lag + 1)]

            self.generate_equations(t, X, U, trends, seasonality, n_vars, max_lag, timestamps, lag_indices)

        # Generate and apply missing data mask
        missing_mask = self.generate_missing_mask(n_points, n_vars)
        X_missing = X.copy()
        X_missing[missing_mask] = np.nan

        # Create DataFrames
        columns = [f'X{i+1}' for i in range(n_vars)]
        df_missing = pd.DataFrame(X_missing, columns=columns)
        df_missing['U'] = U  # Add confounder column
        df_missing['time'] = timestamps

        df_complete = pd.DataFrame(X, columns=columns)
        df_complete['U'] = U
        df_complete['time'] = timestamps

        return df_missing, df_complete, missing_mask

def save_dataset_and_graph(df_missing, df_complete, missing_mask,
                           n_vars, max_lag, sample_size,
                           noise_mix_ratio, mcar_prob, block_prob,
                           output_dir="output_nonlinear_mixed_missing_confounded"):
    # """Save dataset and create causal graph visualization"""
    Path(output_dir).mkdir(parents=True, exist_ok=True)
    base_filename = f'{output_dir}/nonlinear_ts_n{sample_size}_vars{n_vars}_lag{max_lag}_noise{int(noise_mix_ratio*100)}_mcar{int(mcar_prob*100)}_block{int(block_prob*100)}'

    # Save datasets
    df_missing.to_csv(f'{base_filename}_missing.csv', index=False)
    df_complete.to_csv(f'{base_filename}_complete.csv', index=False)

    # Get equations and extract links
    equations = get_nonlinear_equations_with_confounder(n_vars, max_lag)
    true_links = extract_causal_links(equations)

    # Create matrices for tigramite plotting
    var_names = [f'X{i+1}' for i in range(n_vars)] + ['U']
    n_total_vars = n_vars + 1
    val_matrix = np.zeros((n_total_vars, n_total_vars, max_lag + 1))
    graph_matrix = np.zeros((n_total_vars, n_total_vars, max_lag + 1), dtype='bool')

    # Fill matrices based on true links
    for (source, lag, target, func), weight in true_links.items():
        if source == 'U':
            source_idx = n_vars
        else:
            source_idx = int(source[1:]) - 1
        target_idx = int(target[1:]) - 1
        lag_idx = abs(lag)

        # Add the link to the matrices
        val_matrix[source_idx, target_idx, lag_idx] = weight
        graph_matrix[source_idx, target_idx, lag_idx] = True

        # For contemporaneous links, make val_matrix symmetric
        if lag == 0:
            val_matrix[target_idx, source_idx, lag_idx] = weight

    # Plot and save causal graph
    plt.figure(figsize=(12, 12))
    tp.plot_time_series_graph(
        val_matrix=val_matrix,
        graph=graph_matrix,
        var_names=var_names,
        link_colorbar_label='Nonlinear Effect Strength',
        node_size=0.05
    )
    plt.title(f'Nonlinear Causal Graph with Confounder\n(n={sample_size}, vars={n_vars}, lag={max_lag})\n'
              f'Noise: {int(noise_mix_ratio*100)}% Gaussian, {int((1-noise_mix_ratio)*100)}% Laplace\n'
              f'MCAR: {int(mcar_prob*100)}%, Block: {int(block_prob*100)}%')
    plt.savefig(f'{base_filename}_graph.png')
    plt.close()

    # Plot time series with mixed missing data and confounder
    plt.figure(figsize=(15, 10))
    for col in df_missing.columns[:-2]:  # Exclude U and time
        plt.plot(df_complete['time'], df_complete[col], label=f'{col} (complete)', alpha=0.3)
        mask = ~df_missing[col].isna()
        plt.scatter(df_missing.loc[mask, 'time'], df_missing.loc[mask, col],
                   label=f'{col} (observed)', alpha=0.7, s=20)

    # Plot confounder U
    plt.plot(df_complete['time'], df_complete['U'], label='U (confounder)',
             linestyle='--', color='black', alpha=0.7)

    plt.title(f'Nonlinear Time Series with Mixed Missing Data and Confounder\n'
              f'(n={sample_size}, vars={n_vars}, lag={max_lag})\n'
              f'Noise: {int(noise_mix_ratio*100)}% Gaussian, {int((1-noise_mix_ratio)*100)}% Laplace\n'
              f'MCAR: {int(mcar_prob*100)}%, Block: {int(block_prob*100)}%')
    plt.xlabel('Time')
    plt.ylabel('Value')
    plt.legend()
    plt.grid(True)
    plt.savefig(f'{base_filename}_series.png')
    plt.close()

    # Save missing data pattern visualization
    plt.figure(figsize=(15, 5))
    plt.imshow(missing_mask.T, aspect='auto', cmap='binary')
    plt.title(f'Missing Data Pattern (black = missing)\n'
              f'Noise: {int(noise_mix_ratio*100)}% Gaussian, {int((1-noise_mix_ratio)*100)}% Laplace\n'
              f'MCAR: {int(mcar_prob*100)}%, Block: {int(block_prob*100)}%')
    plt.xlabel('Time')
    plt.ylabel('Variable')
    plt.yticks(range(n_vars), [f'X{i+1}' for i in range(n_vars)])
    plt.colorbar(label='Missing')
    plt.savefig
    (f'{base_filename}_missing_pattern.png')
    plt.close()

    # Plot sampling intervals distribution
    plt.figure(figsize=(10, 5))
    time_diffs = np.diff(df_complete['time'])
    plt.hist(time_diffs, bins=50, density=True)
    plt.title('Distribution of Sampling Intervals')
    plt.xlabel('Time Difference')
    plt.ylabel('Density')
    plt.grid(True)
    plt.savefig(f'{base_filename}_sampling.png')
    plt.close()

    # Save causal structure description
    with open(f'{base_filename}_structure.txt', 'w') as f:
        f.write("Nonlinear Causal Structure with Mixed Errors, Irregular Sampling, Mixed Missing Data, and Confounder:\n\n")
        f.write(f"Noise: {int(noise_mix_ratio*100)}% Gaussian, {int((1-noise_mix_ratio)*100)}% Laplace\n")
        f.write(f"MCAR: {int(mcar_prob*100)}%, Block: {int(block_prob*100)}%\n\n")
        f.write("Equations:\n")
        for eq in equations:
            f.write(f"{eq}\n")
        f.write("\nCausal Links:\n")
        for (source, lag, target, func), coef in true_links.items():
            f.write(f"{source} --({func}, lag={lag})--> {target}: {coef}\n")

def generate_all_combinations():
    """Generate datasets for all combinations"""
    sample_sizes = [500, 1000, 3000, 5000]
    n_vars_list = [4, 6, 8]
    max_lags = [2, 3, 4]
    noise_mix_ratios = [0.3, 0.5, 0.7]
    mcar_probs = [0.05, 0.1]
    block_probs = [0.05, 0.1]

    for n in sample_sizes:
        for vars_ in n_vars_list:
            for lag in max_lags:
                for noise_ratio in noise_mix_ratios:
                    for mcar_prob in mcar_probs:
                        for block_prob in block_probs:
                            print(f"\nGenerating dataset: n={n}, vars={vars_}, lag={lag}, "
                                  f"noise_ratio={noise_ratio}, mcar={mcar_prob}, block={block_prob}")

                            generator = NonlinearTimeSeriesGeneratorMixedMissingConfounded(
                                noise_mix_ratio=noise_ratio,
                                noise_params={'scale': 0.1},
                                mcar_prob=mcar_prob,
                                block_size_range=(5, 20),
                                block_prob=block_prob,
                                trend_strength=0.01,
                                seasonal_strength=0.5,
                                seasonal_period=12,
                                random_state=42
                            )

                            df_missing, df_complete, missing_mask = generator.generate_multivariate_ts(
                                n_points=n,
                                n_vars=vars_,
                                max_lag=lag,
                                total_time=100,
                                min_gap=0.1
                            )

                            save_dataset_and_graph(
                                df_missing=df_missing,
                                df_complete=df_complete,
                                missing_mask=missing_mask,
                                n_vars=vars_,
                                max_lag=lag,
                                sample_size=n,
                                noise_mix_ratio=noise_ratio,
                                mcar_prob=mcar_prob,
                                block_prob=block_prob
                            )
                            print("Dataset and visualizations saved successfully")

if __name__ == "__main__":
    print("Generating nonlinear time series with mixed errors, irregular sampling, "
          "mixed missing patterns, and confounder...")
    generate_all_combinations()
    print("\nAll datasets generated successfully!")

Generating nonlinear time series with mixed errors, irregular sampling, mixed missing patterns, and confounder...

Generating dataset: n=500, vars=4, lag=2, noise_ratio=0.3, mcar=0.05, block=0.05
Dataset and visualizations saved successfully

Generating dataset: n=500, vars=4, lag=2, noise_ratio=0.3, mcar=0.05, block=0.1
Dataset and visualizations saved successfully

Generating dataset: n=500, vars=4, lag=2, noise_ratio=0.3, mcar=0.1, block=0.05
Dataset and visualizations saved successfully

Generating dataset: n=500, vars=4, lag=2, noise_ratio=0.3, mcar=0.1, block=0.1
Dataset and visualizations saved successfully

Generating dataset: n=500, vars=4, lag=2, noise_ratio=0.5, mcar=0.05, block=0.05
Dataset and visualizations saved successfully

Generating dataset: n=500, vars=4, lag=2, noise_ratio=0.5, mcar=0.05, block=0.1
Dataset and visualizations saved successfully

Generating dataset: n=500, vars=4, lag=2, noise_ratio=0.5, mcar=0.1, block=0.05
Dataset and visualizations saved successfu

/usr/local/lib/python3.11/dist-packages/tigramite/plotting.py:3203: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig = pyplot.figure(figsize=figsize)
<ipython-input-15-54303aa26c4e>:447: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  plt.figure(figsize=(15, 10))
<ipython-input-15-54303aa26c4e>:470: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly clo

Dataset and visualizations saved successfully

Generating dataset: n=500, vars=4, lag=3, noise_ratio=0.7, mcar=0.05, block=0.05


<ipython-input-15-54303aa26c4e>:432: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  plt.figure(figsize=(12, 12))


Dataset and visualizations saved successfully

Generating dataset: n=500, vars=4, lag=3, noise_ratio=0.7, mcar=0.05, block=0.1
Dataset and visualizations saved successfully

Generating dataset: n=500, vars=4, lag=3, noise_ratio=0.7, mcar=0.1, block=0.05
Dataset and visualizations saved successfully

Generating dataset: n=500, vars=4, lag=3, noise_ratio=0.7, mcar=0.1, block=0.1
Dataset and visualizations saved successfully

Generating dataset: n=500, vars=4, lag=4, noise_ratio=0.3, mcar=0.05, block=0.05
Dataset and visualizations saved successfully

Generating dataset: n=500, vars=4, lag=4, noise_ratio=0.3, mcar=0.05, block=0.1
Dataset and visualizations saved successfully

Generating dataset: n=500, vars=4, lag=4, noise_ratio=0.3, mcar=0.1, block=0.05
Dataset and visualizations saved successfully

Generating dataset: n=500, vars=4, lag=4, noise_ratio=0.3, mcar=0.1, block=0.1
Dataset and visualizations saved successfully

Generating dataset: n=500, vars=4, lag=4, noise_ratio=0.5, mcar=0

...

<Figure size 1200x1200 with 0 Axes>

<Figure size 1200x1200 with 0 Axes>

<Figure size 1200x1200 with 0 Axes>

<Figure size 1200x1200 with 0 Axes>

<Figure size 1200x1200 with 0 Axes>

In [16]:
# Optional: Zip and download output
!zip -r /content/D3C.zip /content/output_nonlinear_mixed_missing_confounded
from google.colab import files
files.download('/content/D3C.zip')

  adding: content/output_nonlinear_mixed_missing_confounded/ (stored 0%)
  adding: content/output_nonlinear_mixed_missing_confounded/nonlinear_ts_n500_vars6_lag4_noise70_mcar5_block5_structure.txt (deflated 55%)
  adding: content/output_nonlinear_mixed_missing_confounded/nonlinear_ts_n500_vars8_lag2_noise50_mcar10_block5_graph.png (deflated 5%)
  adding: content/output_nonlinear_mixed_missing_confounded/nonlinear_ts_n5000_vars4_lag4_noise50_mcar5_block10_graph.png (deflated 4%)
  adding: content/output_nonlinear_mixed_missing_confounded/nonlinear_ts_n1000_vars8_lag2_noise30_mcar10_block10_complete.csv (deflated 51%)
  adding: content/output_nonlinear_mixed_missing_confounded/nonlinear_ts_n500_vars4_lag3_noise70_mcar5_block5_graph.png (deflated 4%)
  adding: content/output_nonlinear_mixed_missing_confounded/nonlinear_ts_n1000_vars8_lag3_noise30_mcar5_block10_missing.csv (deflated 51%)
  adding: content/output_nonlinear_mixed_missing_confounded/nonlinear_ts_n3000_vars6_lag3_noise30_mcar5

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>